# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../output_data/cities.csv")

# Display sample data
city_data_df.head()

City_ID               City      Lat       Lng  Max Temp  Humidity  \
0        0             kodiak  57.7900 -152.4072      3.73        56   
1        1  port-aux-francais -49.3500   70.2167      4.52        77   
2        2     kulhudhuffushi   6.6221   73.0700     29.91        73   
3        3            maymana  35.9214   64.7836     24.37        42   
4        4           kingston  17.9970  -76.7936     29.53        74   

   Cloudiness  Wind Speed Country        Date  
0          75        4.12      US  1715202648  
1          93       10.02      TF  1715202645  
2          90        4.59      MV  1715202706  
3          15        1.34      AF  1715202751  
4          75        2.06      JM  1715201948

---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
city_plot = city_data_df.hvplot.points("Lng", "Lat", geo = True, tiles = "OSM",
                                       frame_width = 700, frame_height = 500,
                                       size = "Humidity", scale = 0.5, color = "City")

# Display the map
city_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
narrowed_city_df = city_data_df.loc[(city_data_df["Max Temp"] < 32) & (city_data_df["Max Temp"] > 26)
                                   & (city_data_df["Wind Speed"] < 6)
                                    & (city_data_df["Cloudiness"] < 10)]


# Drop any rows with null values
filtered_city_df = narrowed_city_df.dropna()

# Display sample data
filtered_city_df

City_ID               City      Lat       Lng  Max Temp  Humidity  \
49        49        kailua-kona  19.6406 -155.9956     26.98        76   
55        55          mananwala  31.5842   73.6878     30.78        14   
67        67             bethel  41.3712  -73.4140     29.47        64   
74        74             hadibu  12.6500   54.0333     29.14        74   
131      131           nuevitas  21.5453  -77.2644     28.33        66   
190      190         al ghayzah  16.2079   52.1760     29.80        73   
203      203             ormara  25.2088   64.6357     27.32        87   
210      210           elk city  35.4120  -99.4043     26.99        27   
251      251          childress  34.4264 -100.2040     28.29        23   
263      263  sao joao da barra -21.6403  -41.0511     26.05        89   
386      386             katpur  21.0500   71.7833     29.32        62   
395      395            udaipur  24.5712   73.6918     29.03        29   
406      406           timimoun  29.2639    0.2310     28.21        13   
449      449             medina  24.4686   39.6142     27.24        12   
539      539           griffith  41.5284  -87.4237     29.06        40   
567      567            salinas  36.6777 -121.6555     28.88        42   
581      581             jiwani  25.0500   61.7417     28.68        69   

     Cloudiness  Wind Speed Country        Date  
49            0        5.14      US  1715202728  
55            0        2.71      PK  1715202777  
67            0        4.63      US  1715202649  
74            1        1.61      YE  1715202787  
131           0        5.38      CU  1715202817  
190           0        4.12      YE  1715202848  
203           0        2.55      PK  1715202855  
210           0        1.54      US  1715202567  
251           0        2.06      US  1715202814  
263           0        4.40      BR  1715202653  
386           0        4.82      IN  1715202947  
395           0        4.68      IN  1715202952  
406           0        5.66      DZ  1715202958  
449           0        1.54      SA  1715202901  
539           0        5.66      US  1715202978  
567           0        4.63      US  1715203019  
581           0        1.85      PK  1715203049

### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = filtered_city_df[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

City Country      Lat       Lng  Humidity Hotel Name
49         kailua-kona      US  19.6406 -155.9956        76           
55           mananwala      PK  31.5842   73.6878        14           
67              bethel      US  41.3712  -73.4140        64           
74              hadibu      YE  12.6500   54.0333        74           
131           nuevitas      CU  21.5453  -77.2644        66           
190         al ghayzah      YE  16.2079   52.1760        73           
203             ormara      PK  25.2088   64.6357        87           
210           elk city      US  35.4120  -99.4043        27           
251          childress      US  34.4264 -100.2040        23           
263  sao joao da barra      BR -21.6403  -41.0511        89           
386             katpur      IN  21.0500   71.7833        62           
395            udaipur      IN  24.5712   73.6918        29           
406           timimoun      DZ  29.2639    0.2310        13           
449             medina      SA  24.4686   39.6142        12           
539           griffith      US  41.5284  -87.4237        40           
567            salinas      US  36.6777 -121.6555        42           
581             jiwani      PK  25.0500   61.7417        69

### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set parameters to search for a hotel
radius = 10000
limit = 5
apiKey = geoapify_key
params = {"apiKey": geoapify_key,
          "limit": limit}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    Lat = hotel_df.loc[index, "Lat"]
    Lng = hotel_df.loc[index, "Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{Lng},{Lat},{radius}"
    params["bias"] = f"proximity:{Lng},{Lat}"
    params["categories"] = f"accommodation.hotel"
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
kailua-kona - nearest hotel: Kona Seaside Hotel
mananwala - nearest hotel: No hotel found
bethel - nearest hotel: Hampton Inn Danbury
hadibu - nearest hotel: No hotel found
nuevitas - nearest hotel: Villa Sabinal (El CECE)
al ghayzah - nearest hotel: فندق تاج العرب
ormara - nearest hotel: No hotel found
elk city - nearest hotel: King's Inn
childress - nearest hotel: Red Roof Inn Childress
sao joao da barra - nearest hotel: Pousada Mediterrâneo
katpur - nearest hotel: No hotel found
udaipur - nearest hotel: Pannadhay Palace
timimoun - nearest hotel: أناف
medina - nearest hotel: The Oberoi Madina
griffith - nearest hotel: Quality Inn & Suites Hammond
salinas - nearest hotel: The Traveler's Hotel
jiwani - nearest hotel: No hotel found


City Country      Lat       Lng  Humidity  \
49         kailua-kona      US  19.6406 -155.9956        76   
55           mananwala      PK  31.5842   73.6878        14   
67              bethel      US  41.3712  -73.4140        64   
74              hadibu      YE  12.6500   54.0333        74   
131           nuevitas      CU  21.5453  -77.2644        66   
190         al ghayzah      YE  16.2079   52.1760        73   
203             ormara      PK  25.2088   64.6357        87   
210           elk city      US  35.4120  -99.4043        27   
251          childress      US  34.4264 -100.2040        23   
263  sao joao da barra      BR -21.6403  -41.0511        89   
386             katpur      IN  21.0500   71.7833        62   
395            udaipur      IN  24.5712   73.6918        29   
406           timimoun      DZ  29.2639    0.2310        13   
449             medina      SA  24.4686   39.6142        12   
539           griffith      US  41.5284  -87.4237        40   
567            salinas      US  36.6777 -121.6555        42   
581             jiwani      PK  25.0500   61.7417        69   

                       Hotel Name  
49             Kona Seaside Hotel  
55                 No hotel found  
67            Hampton Inn Danbury  
74                 No hotel found  
131       Villa Sabinal (El CECE)  
190                فندق تاج العرب  
203                No hotel found  
210                    King's Inn  
251        Red Roof Inn Childress  
263          Pousada Mediterrâneo  
386                No hotel found  
395              Pannadhay Palace  
406                          أناف  
449             The Oberoi Madina  
539  Quality Inn & Suites Hammond  
567          The Traveler's Hotel  
581                No hotel found

### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [7]:
%%capture --no-display

# Configure the map plot
map_plot = hotel_df.hvplot.points("Lng", "Lat", geo = True, tiles = "OSM",
                                  frame_width = 700, frame_height = 500,
                                  size = "Humidity", scale = 0.5, color = "City",
                                  hover_cols = ["Hotel Name", "Country"])

# Display the map
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)